In [22]:
import numpy as np
indices = np.random.choice(10, 5, replace=True)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(arr[indices])
print(arr[-11:])
reward_buffer = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1]
size = 7
game_start_index = 0
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
game_start_index = 7
size = 15
print(len(reward_buffer))
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
print(len(reward_buffer))
overflow_reward = 1
overflow_count = 1

[ 2  3 10  8  2]
[ 1  2  3  4  5  6  7  8  9 10]
[ 1 -1  1 -1  1 -1  1]
[1, -1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1]
15
[ 1 -1  1 -1  1 -1  1 -1]
[1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1]
15


In [25]:
reward_buffer = np.array([1, -1, 1, -1, 1, -1, 0])
reward_buffer[6] = [1, 1, 1, 1]
print(reward_buffer)

ValueError: setting an array element with a sequence.

In [27]:
from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf

env = gym.make('ALE/MsPacman-v5', render_mode='rgb_array')

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # 'n_step': 3,
        # REWARD CLIPPING
        'num_epochs': 30,
    }    

agent = AlphaZeroAgent(env, config=config)

1 Physical GPUs, 1 Logical GPUs


NamespaceNotFound: Namespace environments not found. Have you installed the proper package for environments?